In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Import the necessary modules
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

In [ ]:
# New data set with 'cambio_matricula' column
x = pd.read_csv('https://raw.githubusercontent.com/edghero/data/main/escuelas_nuevas.csv').dropna()

In [ ]:
x['cambio_matricula'] = pd.to_numeric(x['cambio_matricula'], errors = 'coerce')

In [ ]:
# Setting index
x = x.set_index('escuela')

In [ ]:
# renaming nivel to match DE's current naming convention. I NEED TO GO OVER INTERLOCKING ONE BY ONE TO RECLASS THEM PROPERLY
x['nivel'] = np.where(x['nivel'] == 'elemental', 'primario',
                      np.where(x['nivel'] == 'intermedia','primario',
                               np.where(x['nivel'] == 'especializada','todos los niveles',
                                        np.where(x['nivel'] == 'superior', 'secundario',
                                                 np.where(x['nivel'] == 'interlocking','todos los niveles','otro')))))

In [ ]:
x = x[['region','nivel','consolidada','matricula','cambio_matricula','promedio_espanol','promedio_matematica','promedio_ingles','promedio_ciencias']]

In [ ]:
# Filtering by escuelas nivel primario
primario = x[x['nivel'] == 'primario'].drop(columns = 'nivel')

In [ ]:
primario = pd.get_dummies(primario, prefix_sep = "_").drop(columns = 'region_SAN JUAN')

In [ ]:
X = primario.drop(columns = 'consolidada')
y = primario['consolidada']

In [ ]:
# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

# Create the classifier: logreg
logreg = LogisticRegression()

# Fit the classifier to the training data
logreg.fit(X_train, y_train)

# Predict the labels of the test set: y_pred
y_pred = logreg.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[117  22]
 [ 22  35]]
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       139
           1       0.61      0.61      0.61        57

    accuracy                           0.78       196
   macro avg       0.73      0.73      0.73       196
weighted avg       0.78      0.78      0.78       196



In [ ]:
# Compute predicted probabilities: y_pred_prob
y_pred_prob = logreg.predict_proba(X_test)[:,1]

In [ ]:
# Compute and print AUC score
print("AUC: {}".format(roc_auc_score(y_test, y_pred_prob)))

AUC: 0.8421052631578947


In [ ]:
# Create the classifier: logreg
logreg = LogisticRegression()

# Fit the classifier to the training data
logreg.fit(X, y)

# Predict the labels of the test set: y_pred
y_pred = logreg.predict(X)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
# Compute and print the confusion matrix and classification report
print(confusion_matrix(y, y_pred))
print(classification_report(y, y_pred))

[[377  66]
 [ 83 127]]
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       443
           1       0.66      0.60      0.63       210

    accuracy                           0.77       653
   macro avg       0.74      0.73      0.73       653
weighted avg       0.77      0.77      0.77       653



In [ ]:
prediction = logreg.predict_proba(X)[:,1]


In [ ]:
primario['prediction'] = prediction

In [ ]:
false_negative_schools_primario = primario[(primario['consolidada'] == 1) & (primario['prediction'] <= .50)]

In [ ]:
false_negative_schools_primario[false_negative_schools_primario['prediction'] >=.40].describe()

,consolidada,matricula,cambio_matricula,promedio_espanol,promedio_matematica,promedio_ingles,promedio_ciencias,region_ARECIBO,region_BAYAMON,region_CAGUAS,region_HUMACAO,region_MAYAGUEZ,region_PONCE,prediction
count,33.0,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,1.0,179.333333,-0.007273,55.424242,56.030303,46.090909,64.303030,0.090909,0.090909,0.060606,0.090909,0.242424,0.212121,0.453950
std,0.0,19.210457,0.251623,22.566610,27.083765,26.392901,27.700953,0.291937,0.291937,0.242306,0.291937,0.435194,0.415149,0.029203
min,1.0,130.000000,-0.340000,13.000000,1.000000,3.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.402162
25%,1.0,164.000000,-0.150000,38.000000,41.000000,27.000000,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.430448
50%,1.0,181.000000,-0.090000,53.000000,54.000000,38.000000,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.454909
75%,1.0,188.000000,0.030000,76.000000,72.000000,65.000000,89.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.480042
max,1.0,227.000000,0.790000,93.000000,100.000000,94.000000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.498356


In [ ]:
false_negative_schools_primario[false_negative_schools_primario['prediction'] <=.10].describe()

,consolidada,matricula,cambio_matricula,promedio_espanol,promedio_matematica,promedio_ingles,promedio_ciencias,region_ARECIBO,region_BAYAMON,region_CAGUAS,region_HUMACAO,region_MAYAGUEZ,region_PONCE,prediction
count,5.0,5.0000,5.000000,5.000000,5.000000,5.000000,5.000000,5.0,5.000000,5.0,5.000000,5.000000,5.000000,5.000000
mean,1.0,389.2000,0.114000,44.000000,37.600000,41.400000,46.600000,0.0,0.400000,0.0,0.200000,0.200000,0.200000,0.048120
std,0.0,116.6349,0.447806,28.363709,36.773632,28.130055,32.462286,0.0,0.547723,0.0,0.447214,0.447214,0.447214,0.027687
min,1.0,310.0000,-0.130000,22.000000,2.000000,15.000000,17.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.001502
25%,1.0,336.0000,-0.110000,24.000000,7.000000,21.000000,28.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.044979
50%,1.0,341.0000,-0.100000,34.000000,31.000000,37.000000,36.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.058984
75%,1.0,364.0000,0.000000,49.000000,58.000000,48.000000,52.000000,0.0,1.000000,0.0,0.000000,0.000000,0.000000,0.065485
max,1.0,595.0000,0.910000,91.000000,90.000000,86.000000,100.000000,0.0,1.000000,0.0,1.000000,1.000000,1.000000,0.069650


In [ ]:
false_negative_schools_primario[false_negative_schools_primario['prediction'] <= .10]

,consolidada,matricula,cambio_matricula,promedio_espanol,promedio_matematica,promedio_ingles,promedio_ciencias,region_ARECIBO,region_BAYAMON,region_CAGUAS,region_HUMACAO,region_MAYAGUEZ,region_PONCE,prediction
escuela,,,,,,,,,,,,,,
Abraham Lincoln,1,341,-0.11,49.0,58.0,37.0,52.0,0,1,0,0,0,0,0.065485
Rexville Intermedia,1,364,-0.10,24.0,2.0,21.0,17.0,0,1,0,0,0,0,0.058984
Fulgencio Piñero Rodriguez,1,336,0.00,22.0,31.0,15.0,36.0,0,0,0,1,0,0,0.044979
Antonio S Pedreira,1,595,0.91,34.0,7.0,48.0,28.0,0,0,0,0,1,0,0.001502
Walter Mc Jones,1,310,-0.13,91.0,90.0,86.0,100.0,0,0,0,0,0,1,0.069650


In [ ]:
false_negative_schools_primario.describe()

,consolidada,matricula,cambio_matricula,promedio_espanol,promedio_matematica,promedio_ingles,promedio_ciencias,region_ARECIBO,region_BAYAMON,region_CAGUAS,region_HUMACAO,region_MAYAGUEZ,region_PONCE,prediction
count,83.0,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000
mean,1.0,225.734940,0.030000,52.927711,52.602410,44.554217,60.734940,0.132530,0.108434,0.096386,0.144578,0.144578,0.168675,0.317341
std,0.0,63.875085,0.230106,22.853608,29.419149,26.552412,26.766417,0.341127,0.312818,0.296913,0.353813,0.353813,0.376741,0.138038
min,1.0,130.000000,-0.340000,13.000000,1.000000,3.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001502
25%,1.0,184.000000,-0.110000,34.500000,32.000000,25.000000,44.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.201942
50%,1.0,213.000000,-0.010000,53.000000,53.000000,38.000000,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.344260
75%,1.0,259.000000,0.100000,66.000000,72.000000,63.500000,84.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.443567
max,1.0,595.000000,0.910000,100.000000,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.498356
